<a href="https://colab.research.google.com/github/br4bit/Neural-Network-Training/blob/master/Guida_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Fase di costruzione del modello



1. Importare la libreria di *tensorflow* e il dataset del *MNIST*
2. Bisogna specificare il numero di inputs,outputs e il numero di hidden neurons in ogni layers.

Il dataset su cui andremo ad operare è il *MNIST* dove ogni immagine è 28x28, quindi il numero dei nostri neuroni di inputs sarà 784.

Numero di neuroni nel primo hidden layer sarà 300, mentre nel secondo 100.
Il numero di neuroni di output è determinato dal numero di classi a cui un'immagine può appartenere. Esempio: Nel caso del MNIST dataset le immagini da classificare contengono numeri da 0 a 9.
Quindi il nostro target sarà quello di classificare un'immagine e determinarne l'appartenenza o meno a una delle 10 classi.

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

## 1.1 Fase di organizzazione degli inputs e outputs

Per addestrare una rete neurale c'è bisogno di un training set (X) e di un target set (Y) che contengono rispettivamente i dati utilizzati per addestrare una rete. Nel caso del *MNIST* , nel training set sarà presente un'immagine che conterrà un numero da 0 a 9 e nel target set sarà presente un numero in corrispondenza di quella immagine che indicherà la classe di appartenenza.

Primo problema: La grandezza delle immagini è 28x28 (one feature per pixel), ma non si conosce ancora quante instanze(campioni) il *batch* conterrà (32,64,128,ecc). Lo *shape* di X sarà (None,n_inputs). X non è nient'altro che una matrice di dimensione (Numero_di_campioni X Numero_di_features(in questo caso sono i pixel(28x28))).

Secondo problema: Y sarà un vettore di dimensione 1 (1D Tensor) con un numero di elementi pari al numero di campioni, ma non si conosce ancora la grandezza del *training batch* , quindi la sua dimensione (shape) è None.

Con il termine *training batch* si indica la grandezza del *batch* ovvero il numero di campioni con cui sarà addestrata la rete, che anzichè utilizzarlo interamente sarà splittato in tanti piccoli *batch* di grandezza fissa in modo da ottimizzare il processo di calcolo del gradiente. (Molto più veloce rispetto a quando è addestrata sull'intero training set).



In [0]:
X = tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y = tf.placeholder(tf.int64,shape=(None),name="y")

## 1.2 Fase di creazione dei layers (1 versione)
Ora c'è bisogno di crerare i due hidden layers e l'output layer.

I due hidden layers sono quasi identici: differiscono solo per il numero di inputs a cui sono connessi e il numero di neuroni che contengono (300 e 100).

L'output layer è molto simile, ma invece di utilizzare la funzione **ReLU** come funzione di attivazione, utilizza la **Softmax** (per la multiclassificazione, il nostro vettore di ouput y_predict conterrà le probabilità di appartenere a ciascuna delle 10 classi).

Definiamo la funzione neuron_layer() , usata per la creazione di un layer alla volta. Avrà bisogno di alcuni parametri per specificare gli inputs, il numero di neuroni, e la funzione di attivazione, seguito dal nome del layer.


In [0]:
def neuron_layer(X, n_neuroni, nome, activation=None):
  with tf.name_scope(nome):
    n_inputs=int(X.get_shape()[1])
    stddev= 2 / np.sqrt(n_inputs + n_neuroni) #1
    init=tf.truncated_normal((n_inputs, n_neuroni),stddev=stddev)#2
    W = tf.Variable(init, name="kernel") #3
    b = tf.Variable(tf.zeros([n_neuroni]),name="bias")#4
    Z = tf.matmul(X,W) + b # Z = X*W + b
    if activation is not None:
      return activation(Z)
    else:
      return Z
    

Andiamo ad analizzare questa funzione linea per linea:


1.   Per prima cosa creiamo uno scope di nomi usando il nome del layer, conterrà tutti i calcoli dei nodi per quel layer. E' usato per rendere più leggibile il grafico all'interno di TensorBoard.

2. Il numero di inputs del layer: nel primo layer ci sono 300 neuroni e il numero di inputs sarà dato dal numero di features della matrice X o meglio il numero di colonne quindi la sua seconda dimensione, la prima dimensione è per le instanze(campioni).**Attenzione: Nella creazione del secondo layer, il numero di inputs non sarà più uguale al numero di colonne della matrice X, ma sarà uguale al numero di neuroni del layer che lo precede.**

*Nota: E' possibile vedere gli inputs (features(colonne) di X) come un layer (layer di inputs) dove il numero di neuroni in questo caso è pari al numero di features della matrice X, ragionando in questo modo il numero di inputs del primo hidden layer è pari al numero di neuroni(inputs) del layer che lo precede(layer di inputs). Quindi anche nell'ultimo layer(output layer), il numero di inputs sarà pari al numero di neuroni del layer precedente(hidden2_layer).*.

3. Le righe commentate da #1 a #3 sono utilizzate per la creazione della matrice dei pesi W (layer's kernel). Sarà un 2D tensor contenente tutti i pesi delle connessioni tra gli input e ogni neurone, la sua dimensione sarà (n_inputs x n_neuroni). Questa matrice è inizializzata con valori ranndom utilizzando una distribuzione gaussiana normale con una deviazione standard 2/sqrt(n_inputs+n_neuroni). Utilizzando questa specifica deviazione standard sarà di aiuto all'algoritmo del calcolo del gradiente, di convergere più velocemente al minimo globale. E' molto importante che la matrice dei pesi W per ogni hidden layer sia inizializzata cpn valori casuali, per evitare qualsiasi simmetrie che l'algoritmo del Gradiente Descent non sarebbe in grado di rompere. In parole povere, bisogna evitare simmetrie tra i pesi della matrice o si rischia nel caso di una Deep Net , di incorrere nel problema del vanishing gradient.

4. La riga successiva crea la variabile b (biases) inizializzata a 0 (non ci sono problemi di simmetria con il bias), serve aggiungere il parametro bias per ogni neurone, quindi la sua dimensione sarà pari a n_neuroni.

5. Crea un subgraph per il calcolo dell'attivazione Z= XW+b. Questa è una versione vettorizzata della formula che efficientemente calcola la somma dei pesi (W) e degli input (X) aggiungendo il termine bias per ogni neurone nel layer, per tutte le istanze del batch in un colpo solo.
*Nota: Aggiungere un vettore di 1D (b) ad una matrice di 2D, con lo stesso numero di righe e colonne (X.W), non è nient'altro che l'aggiunta di un vettore 1D ad ogni riga della matrice: broadcasting*.

6. Nell'ultima parte del codice, se il parametro activation è diverso da None (tf.nn.relu()), allora il codice ritorna activation(Z) o altrimenti solo Z. Nel caso dell'output layer l'activation è None poichè Z sarà l'input della nostra softmax, la funzione ReLU è utilizzata solo nell'activation degli hidden layer, per far si che ci siano valori compresi tra 0 e 1 soltanto.

**Logits è l'output della rete neurale prima di andare in pasto alla funzione di attivazione softmax**

In [0]:
with tf.name_scope("dnn"):
  hidden1 = neuron_layer(X, n_hidden1, nome="hidden1", activation=tf.nn.relu)
  hidden2 = neuron_layer(hidden1, n_hidden2, nome="hidden2", activation=tf.nn.relu)
  logits = neuron_layer(hidden2, n_outputs, nome="output") #ritorna Z

## 1.2 Fase di creazione dei layers (2 versione)

Tensorflow ci viene in aiuto nel creare gli hidden layers, quindi non c'è bisogno di definire una propria funzione neuron_layer(). Ad esempio la funzione *tf.layers.dens()* crea un fully connected layer, dove tutti gli inputs sono connessi ai neuroni del layer e si preoccupa anche di creare il vettore bias del layer, utilizzando l'appropriata strategia di inizializzazione del kernel (W) a valori casuali con una distribuzione normale gaussiana 2/sqrt(n_inputs+n_neuroni), ed è possibile scegliere anche il tipo di funzione di attivazione.

In [0]:
with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
  hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
  logits = tf.layers.dense(hidden2, n_outputs, name="outputs")


# Fine di creazione del modello
---


# 2.Definizione della cost function e loss

La cost function utilizzata è la cross entropy. TensorFlow ha diverse funzioni per il calcolo della cross entropy. Usiamo la *sparse_softmax_cross_entropy_with_logits()* : che calcola la cross entropy basata sull'output della nostra rete neurale *logits* (l'output della rete prima che venga dato in pasto alla sofrmax activation), la funzione si aspetta come parametro i *labels* ovvero i target (Y) del nostro training set (X), che saranno interi da 0 al numero di classi - 1 (nel nostro caso da 0 a 9). Il risultato di questa funzione sarà un vettore di 1D (Tensor), contenente la cross entropy per ogni campione del nostro training set.

Infine con la funzione *tf.reduce_mean()* applicata a 1D Tensor calcolato al punto precedente, si calcola la media della cross entropy su tutte le istanze (campioni). E' l'equivalente della moltiplicazione di 2/m della sommatoria su tutti i campioni del training set applicata alla cross entropy, ovvero il calcolo della media.

In [0]:
with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) #definizione/calcolo cross entropy
  loss = tf.reduce_mean(xentropy, name="loss") #media della cross entropy su tutti i campioni

*Nota: La funzione sparse_softmax_cross_entropy_with_logits() è l'equivalente dell'applicare la softmax all'output logits e calcolare la cross entropy sul tutto il training set, ma è molto più efficiente, e si occupa di alcuni problemi: nel caso in cui logits sia molto elevato, l'errore di arrotondamento della virgola mobile, può causare che l'output della softmax sia esattamente 0 o 1, e in questo caso l'equazione della cross entropy conterrò un termine log(0), uguale all'infinito negativo. La sparse risolve questo problema andando a calcolare log(epsilon), dove epsilon è una quantità positiva molto piccola. E' possibile anche utilizzare un'altra funzione che risolve questo tipo di problema dovuto all'arrotondamento della virgola mobile: softmax_cross_entropy_with_logits(), che riceve come labels y nella forma one-hot vectors (invece che interi compresi tra 0 e numero di classi - 1). Ma richiede un refactoring del vettore y che dovrà essere trasformato in un one-hot vectors*.

## 2.1 Calcolo del Gradient Descent

Ora che abbiamo il modello di rete neurale, la cost function, non ci resta che definire il calcolo del Gradiente che ci aiuterà a modificare i parametri del modello affinchè si minimizzi la funzione di costo.

In [0]:
learning_rate=0.01
with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)

## 2.2 Come valutare il modello

Useremo la precisione (accuracy) come misura della performance sul training set, in poche parole per ogni istanza (campione) presente nella matrice X , determiniamo se il risultato è corretto andando a controllare se il valore più alto del logits corrisponde al valore del target (y) di quel campione.

*in_top_k()* ci aiuterà in questo calcolo. Ritorna un 1D Tensor con valori booleani (true/false), quindi c'è bisogno di convertire questi valori in float in modo da poterne calcolare la media in modo da darci una precisione complessiva della rete.

Infine si crea un nodo per inizilizzare tutte le variabili, e un *Saver* in modo da salvare i parametri del nostro modello addestrato sul disco.

In [0]:
with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(logits,y,1)
  accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
  
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Fine della definizione/calcolo cost function e valutazione del modello

---



# 3.Fase di Esecuzione, addestramento rete

Per prima cosa carichiamo il dataset del *MNIST*, utilizzando quello di TensorFlow abbiamo un aiuto sui dati che sono stati già preprocessati e ci offre una funzione per caricare un mini-batch alla volta senza bisogno di implementare la funzione fetch_batch.
Inoltre i dati sono già divisi nelle 2 categorie: training set (55.000 campioni), validation set (5.000 campioni) e test set (10.000 campioni).

In [0]:
mnist = input_data.read_data_sets("/tmp/data/")

n_epochs = 50
batch_size = 32

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size): #55000 / 50 = 1100 = numero totale dei mini-batch
      X_batch,y_batch = mnist.train.next_batch(batch_size)
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
    acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
    print(epoch, "Train Accuracy:", acc_train, "Val Accuracy:", acc_val)
    save_path = saver.save(sess,"./my_model_final.ckpt")

Questo codice avvia una sessione di TensorFlow e avvia init node in modo da inizializzare tutte le variabili e avvia il loop principale per l'addestramento:
Per ogni epoch (epoch indica il numero di volte che bisogna attraversare il training set) il codice itera attraverso un numero di mini_batch che corrisponde alla grandezza totale del training set. Ogni mini-batch è processato attraverso il call del metodo *next_batch()*, successivamente è avviata l'operazione di addestramento (sess.run()) che riceve in pasto il mini-batch corrente contenente i dati di input e i quelli di output (targets). Alla fine di ogni epoch il codice valuta il modello sull'ultimo mini-batch e su tutto il validation set e stampa il risultato, salvando i parametri del modello sul disco.

Valultare la precisione sul training set non ha molto senso, l'importante è avere un ottimo risultato sul validation set, in questo caso la precisione è pari al 97%, bisogna anche calcolare la precisione sul test set che ci indica come il modello si comporta su campioni che non ha mai visto prima.

# Fine fase di addestramento
---



# 4.Utilizzo della rete neurale

Carichiamo i parametri memorizzati sul disco al termine dell'addestramento e le nuove immagini che si vogliono classificare, applicando la stessa modifica delle features che è stata applicata sui training data (scales da 0 a 1).
Infine il codice valuta il logits node (output node) dandogli in pasto le nuove immagini. Se si vogliono conoscere tutte le probabilità stimate delle classi a cui appartengono bisogna applicare la softmax() a logits, ma se si vuole soltanto predire se l'immagine appartiene o meno ad una classe, semplicemente , bisogna prendere dal tensore Z , la classe che ha il valore (logit) più alto, applicando la funzione argmax() che ritorna nient'altro che l'indice del tensor che ha il valore più alto (probabilità più alta).

*Nota: Nel codice sottostante è stato utilizzato il test set (set di immagini che il modello non ha mai visto prima) per vederne il comportamento e la generalizzazione, infatti il modello ha archiviato una precisione del 98% su dati che non ha mai visto.*

In [0]:
with tf.Session() as sess:
  
  saver.restore(sess,"./my_model_final.ckpt")
  
  X_test = np.expand_dims(mnist.test.images[2],axis=0) #(784,?) equivalente x[np.newaxis,:]
  
  Z = logits.eval(feed_dict={X: X_test})
  
  y_pred = np.argmax(Z)
  
  acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
  
print("Test set Accuracy: ",np.round(acc_test*100,3))
print("Value predicted of 2nd instance: ",y_pred)
print("Value labels of 2nd instance :",mnist.test.labels[2])